In [1]:
from contrans import contrans
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import numpy as np
import pandas as pd
import json, os

In [4]:
ct = contrans()

In [7]:
dbserver, engine = ct.connect_to_postgres(os.getenv("POSTGRES_PASSWORD"), create_contrans=True)

In [8]:
members = ct.get_bioguideIDs()
members = ct.make_cand_table(members)
terms, members = ct.terms_df(members)
ideology = ct.get_ideology()

In [9]:
ct.make_members_df(members, ideology)

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,partyletter,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,L000397,18,"Lofgren, Zoe",Democratic,CA,2024-10-22T14:54:19Z,https://api.congress.gov/v3/member/L000397?for...,Image courtesy of the Member,https://www.congress.gov/img/member/671024d7ec...,D,...,NaN,-0.398,-0.409,-87.19115,0.91853,1026.0,33.0,NaN,-0.309,-0.297
1,L000604,04,"Lopez, Greg",Republican,CO,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,R,...,NaN,0.868,0.497,-13.87010,0.87842,107.0,6.0,NaN,0.807,0.567
2,M001225,15,"Mullin, Kevin",Democratic,CA,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,D,...,NaN,-0.476,-0.023,-42.71200,0.95887,1017.0,16.0,NaN,-0.478,-0.045
3,M001229,10,"McIver, LaMonica",Democratic,NJ,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000
4,H001097,S,"Helmy, George S.",Democratic,NJ,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,B001267,S,"Bennet, Michael F.",Democratic,CO,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,B001243,S,"Blackburn, Marsha",Republican,TN,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,B001236,S,"Boozman, John",Republican,AR,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,R,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,B001230,S,"Baldwin, Tammy",Democratic,WI,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
myquery = """
SELECT * FROM members
"""
pd.read_sql_query(myquery, engine)

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,partyletter,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,L000397,18,"Lofgren, Zoe",Democratic,CA,2024-10-22T14:54:19Z,https://api.congress.gov/v3/member/L000397?for...,Image courtesy of the Member,https://www.congress.gov/img/member/671024d7ec...,D,...,None,-0.398,-0.409,-87.19115,0.91853,1026.0,33.0,None,-0.309,-0.297
1,L000604,04,"Lopez, Greg",Republican,CO,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,R,...,None,0.868,0.497,-13.87010,0.87842,107.0,6.0,None,0.807,0.567
2,M001225,15,"Mullin, Kevin",Democratic,CA,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,D,...,None,-0.476,-0.023,-42.71200,0.95887,1017.0,16.0,None,-0.478,-0.045
3,M001229,10,"McIver, LaMonica",Democratic,NJ,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,0.000,0.000
4,H001097,S,"Helmy, George S.",Democratic,NJ,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,B001267,S,"Bennet, Michael F.",Democratic,CO,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
535,B001243,S,"Blackburn, Marsha",Republican,TN,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,R,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
536,B001236,S,"Boozman, John",Republican,AR,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,R,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
537,B001230,S,"Baldwin, Tammy",Democratic,WI,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [4]:
useragent = ct.get_useragent()
useragent

'python-requests/2.32.3'

In [5]:
headers = ct.make_headers(useragent)

In [6]:
members = ct.get_bioguideIDs()

In [8]:
members = ct.make_cand_table()
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item,partyletter,DistIDRunFor,lastname,firstname,name2,CID
0,M001225,15,"Mullin, Kevin",Democratic,CA,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star...",D,CA15,Mullin,Kevin,Mullin (D),N00049438
1,R000599,25,"Ruiz, Raul",Democratic,CA,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...,"[{'chamber': 'House of Representatives', 'star...",D,CA25,Ruiz,Raul,Ruiz (D),N00033510
2,B001288,S,"Booker, Cory A.",Democratic,NJ,2024-08-21T12:42:12Z,https://api.congress.gov/v3/member/B001288?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001288_20...,"[{'chamber': 'Senate', 'startYear': 2013}]",D,NJS,Booker,Cory A.,Booker (D),N00035267
3,K000399,02,"Kiggans, Jennifer A.",Republican,VA,2024-08-05T15:15:39Z,https://api.congress.gov/v3/member/K000399?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66b0ce45b0...,"[{'chamber': 'House of Representatives', 'star...",R,VA02,Kiggans,Jennifer A.,Kiggans (R),N00048315
4,S001159,10,"Strickland, Marilyn",Democratic,WA,2024-07-31T18:36:16Z,https://api.congress.gov/v3/member/S001159?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001159_20...,"[{'chamber': 'House of Representatives', 'star...",D,WA10,Strickland,Marilyn,Strickland (D),N00046320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,B001267,S,"Bennet, Michael F.",Democratic,CO,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]",D,COS,Bennet,Michael F.,Bennet (D),N00030608
481,B001243,S,"Blackburn, Marsha",Republican,TN,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY...",R,TNS,Blackburn,Marsha,Blackburn (R),N00003105
482,B001236,S,"Boozman, John",Republican,AR,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY...",R,ARS,Boozman,John,Boozman (R),N00013873
483,B001230,S,"Baldwin, Tammy",Democratic,WI,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY...",D,WIS,Baldwin,Tammy,Baldwin (D),N00004367


In [10]:
ideology = ct.get_ideology()
ideology

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,118,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",...,NaN,0.379,0.380,-156.79628,0.85447,997,78,NaN,0.453,0.351
1,118,House,21102,41,7,AL,100,NaN,NaN,"SEWELL, Terri",...,NaN,-0.401,0.397,-47.88507,0.95124,958,18,NaN,-0.524,0.369
2,118,House,21500,41,6,AL,200,NaN,NaN,"PALMER, Gary James",...,NaN,0.678,0.114,-83.65844,0.91990,1002,41,NaN,0.673,0.143
3,118,House,22108,41,1,AL,200,NaN,NaN,"CARL, Jerry L.",...,NaN,0.550,0.236,-123.17499,0.88593,1017,56,NaN,0.592,0.214
4,118,House,22140,41,2,AL,200,NaN,NaN,"MOORE, Barry",...,NaN,0.639,-0.244,-93.07810,0.91246,1016,40,NaN,0.709,-0.311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,118,House,21970,25,1,WI,200,NaN,NaN,"STEIL, Bryan",...,NaN,0.417,0.061,-116.26627,0.89167,1014,45,NaN,0.441,0.164
444,118,House,21989,25,7,WI,200,NaN,NaN,"TIFFANY, Thomas P.",...,NaN,0.639,-0.248,-87.08229,0.91755,1012,33,NaN,0.664,-0.286
445,118,House,22115,25,5,WI,200,NaN,NaN,"FITZGERALD, Scott",...,NaN,0.598,0.082,-94.70099,0.91092,1015,45,NaN,0.579,-0.017
446,118,House,22370,25,3,WI,200,NaN,NaN,"VAN ORDEN, Derrick",...,NaN,0.376,-0.012,-184.91463,0.82994,992,83,NaN,0.378,-0.020


In [11]:
votes = ct.get_votes()
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [5]:
members = ct.get_bioguideIDs()
terms, members = ct.terms_df(members)
members = ct.make_cand_table(members)

In [6]:
ideology = ct.get_ideology()
votes = ct.get_votes()

In [23]:
replace_map = {
               "Republican": "R",
               "Democratic": "D",
               "Independent": "I"
              }
members["partyletter"] = members["partyName"].apply(lambda partyname: replace_map[partyname])

In [24]:
members["lastname"] = [name.split(",")[0] for name in members["name"]]
members["firstname"] = [name.split(",")[1] for name in members["name"]]
members["name2"] = [firstname + " " + lastname + " " + f"({partyletter})" for firstname, lastname, partyletter in zip(members["firstname"], members["lastname"], members["partyletter"])]

In [ ]:
cands = pd.read_csv('data/cands22.txt', quotechar="|", header=None)

In [27]:
crosswalk = pd.merge(members, cands, left_on="name2", right_on="FirstLastP", 
                     how="outer", indicator="matched", validate="one_to_one")

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

In [17]:
cands.columns = ['Cycle', 'FECCandID', 'CID','FirstLastP',
                 'Party','DistIDRunFor','DistIDCurr',
                 'CurrCand','CycleCand','CRPICO','RecipCode','NoPacs']

In [ ]:
cands = pd.read_csv('data/cands22.txt', quotechar="|", header=None)

In [18]:
cands

,Cycle,FECCandID,CID,FirstLastP,Party,DistIDRunFor,DistIDCurr,CurrCand,CycleCand,CRPICO,RecipCode,NoPacs
0,2022,H0AK00105,N00039029,Thomas Lamb (3),3,AK01,,,,,3N,
1,2022,H0AL01055,N00044245,Jerry Carl (R),R,AL01,AL01,Y,Y,I,RW,
2,2022,H0AL01063,N00044288,Wes Lambert (R),R,AL01,,,,,RN,
3,2022,H0AL01097,N00044750,James Averhart (D),D,AL01,,,,,DN,
4,2022,H0AL02087,N00030768,Martha Roby (R),R,AL02,,,,,RN,
...,...,...,...,...,...,...,...,...,...,...,...,...
8923,2022,S8WI00224,N00041929,Leah Vukmir (R),R,WIS1,,,,,RN,
8924,2022,S8WV00119,N00041293,Paula Jean Swearengin (D),D,WVS2,,,,,DN,
8925,2022,S8WV00127,N00035531,Evan Jenkins (R),R,WVS1,,,,,RN,
8926,2022,S8WV00135,N00041474,Bo Copley (R),R,WVS1,,,,,RN,


In [7]:
goodbioguide = ct.get_bioguide(name="good", state="Virginia", district=5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."


In [8]:
goodid = goodbioguide["bioguideId"][0]

In [9]:
goodid

'G000595'

In [10]:
goodbills = ct.get_sponsoredlegislation(goodid)

In [11]:
import requests

r = requests.get(goodbills[0]["url"], params = {"api_key": ct.congresskey})
texturl = json.loads(r.text)["bill"]["textVersions"]["url"]

r = requests.get(texturl, params = {"api_key": ct.congresskey})
toscrape = json.loads(r.text)["textVersions"][0]["formats"][0]["url"]

NameError: name 'goodbills' is not defined

In [ ]:
from bs4 import BeautifulSoup

r = requests.get(to_scrape)
mysoup = BeautifulSoup(r.text, "html.parser")
mysoup

In [ ]:
def get_billdata(billurl):
    r = requests.get(billurl, params = {"api_key": ct.congresskey})
    bill_json = json.loads(r.text)
    texturl = bill_json["bill"]["textVersions"]["url"]
    
    r = requests.get(texturl, params = {"api_key": ct.congresskey})
    toscrape = json.loads(r.text)["textVersions"][0]["formats"][0]["url"]

    r = requests.get(toscrape)
    mysoup = BeautifulSoup(r.text, "html.parser")
    billtext = mysoup.text
    bill_json["bill_text"] = billtext

    return bill_json

### Sponsored Legislation

In [12]:
import requests 

root = "https://api.congress.gov/v3"
endpoint = f"/member/{goodid}/sponsored-legislation"
header = ct.make_headers()

params = {"api_key": ct.congresskey,
          "limit": 250}

r = requests.get(root + endpoint, 
                    params=params, 
                    headers=headers)


In [15]:
json.loads(r.text)["sponsoredLegislation"][2]

{'congress': 118,
 'introducedDate': '2024-05-23',
 'latestAction': {'actionDate': '2024-07-05',
  'text': 'Placed on the Union Calendar, Calendar No. 474.'},
 'number': '8534',
 'policyArea': {'name': None},
 'title': 'Protecting Student Athletes’ Economic Freedom Act of 2024',
 'type': 'HR',
 'url': 'https://api.congress.gov/v3/bill/118/hr/8534?format=json'}

In [14]:
house

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [16]:
house = house[["rollnumber", "icpsr", "cast_code"]]

In [17]:
house_mat = pd.merge(house, house, on="rollnumber")

In [19]:
len(house_mat)

211856536

In [20]:
house_mat["agree"] = house_mat["cast_code_x"] == house_mat["cast_code_y"]

In [22]:
house_mat = house_mat.groupby(["icpsr_x", "icpsr_y"]).agg({"agree": "mean"})

In [23]:
house_mat = house_mat.reset_index()

In [24]:
house_mat

,icpsr_x,icpsr_y,agree
0,14854,14854,1.000000
1,14854,14863,0.830948
2,14854,14873,0.386404
3,14854,15029,0.412343
4,14854,15433,0.366726
...,...,...,...
200607,91980,31102,0.731664
200608,91980,31103,0.190805
200609,91980,39301,0.236136
200610,91980,39307,0.239714


In [25]:
ideo = ct.get_ideology()

In [26]:
members = ideo[["bioname", "icpsr", "party_code"]]

In [27]:
members

,bioname,icpsr,party_code
0,"ROGERS, Mike Dennis",20301,200
1,"SEWELL, Terri",21102,100
2,"PALMER, Gary James",21500,200
3,"CARL, Jerry L.",22108,200
4,"MOORE, Barry",22140,200
...,...,...,...
443,"STEIL, Bryan",21970,200
444,"TIFFANY, Thomas P.",21989,200
445,"FITZGERALD, Scott",22115,200
446,"VAN ORDEN, Derrick",22370,200
